In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
# import os
# module_a_dir = os.path.dirname(os.path.abspath(__file__))
# utils_path = os.path.join(module_a_dir, '..', 'Utils')
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import validation_data as Val
import var_A_x_B as vAB
import utils as uti

sys.path.append(workdir_ + 'PyRegridding/Utils/')
import MakePressures as MkP
import GridUtils as GrU


sys.path.append(workdir_ + 'PyRegridding/Drivers/')
import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
%%time

# /glade/campaign/cgd/amp/juliob/ERA5/ne30np4/L135/ERA5_x_ne30np4_L135_rgC2_WO.2000-01-01-00000.nc

Dst='fv1x1'

Hgrid, Hkey, Vgrid ='ne30np4','c','L135'  #'L135'
Src=Hgrid

year,month,day,hour = 1985,1,1,0
second = hour  * 3600
ymdPat=str( year ).zfill(4) + '-' + str(month).zfill(2) + '-' + str(day).zfill(2) + '-' + str(second).zfill(5)


pproc1='_cdf5'
pproc2='' #'_TopFix'
pproc3='_aero'

fileN = f'/glade/campaign/cgd/amp/juliob/ERA5/{Hgrid}/{Vgrid}{pproc1}/ERA5_x_{Hgrid}_{Vgrid}{pproc2}_rgC2_WO{pproc3}.{ymdPat}.nc'
print(fileN)
IC=xr.open_dataset( fileN )


In [ ]:
########################################
# Make seasonal means
########################################
season='n/a'
UUa_s,years_a,months_a = IC.U.values.squeeze() , ymdPat, ''
VVa_s,years_a,months_a = IC.V.values.squeeze() , ymdPat, ''
TTa_s,years_a,months_a = IC.T.values.squeeze() , ymdPat, ''
QQa_s,years_a,months_a = IC.Q.values.squeeze() , ymdPat, ''


In [ ]:
########################################
# Make zonal means
########################################
method='C2'
if (Hkey == 'yx' ):
    ########################
    # Option 1 from lat-lon regirrdded data
    ########################
    
    
    
    #ps_x =Dx.PS.values
    lon_a=IC.lon.values
    lat_a=IC.lat.values
    lev_a=IC.lev.values
    zlev_a=-7.0*np.log( lev_a/1_000. )
    # Zonal Av
    UUa_sz=np.average( UUa_s , axis=2 )
    TTa_sz=np.average( TTa_s , axis=2 )

elif (Hkey=='c'):
    ########################
    # Option 2 from unstructured SE data
    ########################
    
    
    DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
    lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
    lev_a=IC.lev.values
    zlev_a=-7.0*np.log( lev_a/1_000. )
    
    RegridObj = RgF.Horz(Src=Src, Dst=Dst )
    UUa_s_xy=RgF.Horz(xfld_Src=UUa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
    TTa_s_xy=RgF.Horz(xfld_Src=TTa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
    
    # Zonal Av
    UUa_sz=np.average( UUa_s_xy , axis=2 )
    TTa_sz=np.average( TTa_s_xy , axis=2 )


In [ ]:
####
# Validation
####

Uval = Val.data(fld='U', ERA5hourly=True, year=year,month=month,day=day,hour=hour ) #, Years='1996' ) #, Years='2000' )
Tval = Val.data(fld='T', ERA5hourly=True, year=year,month=month,day=day,hour=hour ) #, Years='1996' ) #, Years='2000' )

UUv_sz=np.average( Uval['aa'] , axis=2 )
TTv_sz=np.average( Tval['aa'] , axis=2 )

zlev_v = Uval['lev']
lat_v = Uval['lat']

In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))


# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=2,2
# Adding the hatching for the sponge layer
#sponge_layer = np.zeros_like(UUa_sz)
#sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(15, 12))


ulev=np.linspace( -60,140,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=18

titles =['Init. Cond. U','Init. Cond. T','ERA5 U','ERA5 T','',]
plotTop=140

topspace=0.05

n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ,top_whitespace=topspace ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,UUa_sz ,levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,UUa_sz ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
#ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=2
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ,top_whitespace=topspace ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,TTa_sz ,levels=tlev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,TTa_sz ,levels=tlev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')


n=3
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ,top_whitespace=topspace ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=ulev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=ulev, colors='black')
#ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_title( titles[n-1] ,fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')



n=4
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ,top_whitespace=topspace ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, TTv_sz ,levels=tlev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, TTv_sz ,levels=tlev, colors='black')
#ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_title( titles[n-1] ,fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

fig.suptitle( f'IC file={fileN }', fontsize=14)
# Sponge layer hatching
#ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)


#FigName = f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
#print( FigName) 
#Pu.save_link(png_filename = FigName )

